# SANS2D: Q1D for sample and background

In this notebook, we will be reducing a sample and a background measurements to a one-dimensional $I(Q)$.

**Outline:**

- We will begin by loading the data files containing the sample, direct, and background measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Finally, the sample and background measurement will be converted to the $Q$ dimension

In [ ]:
import matplotlib.pyplot as plt
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Define binning resolution

We define here common time-of-flight, wavalength and $Q$ bins for all the measurements.

In [ ]:
tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')

wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141,
                              unit='angstrom')

q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')

## Loading data files

We load the following files:

- The direct beam function for the main detector (gives detector efficiency as a function of wavelength)
- The sample measurement
- The direct measurement: this is the run with the empty sample holder/cuvette
- the background measurement: this is the run with only the solvent which the sample is placed in

In [ ]:
ds = sc.Dataset()

#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

direct_beam = loki.io.load_rkh_wav(
    loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))

ds['sample'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

ds['direct'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

ds['background'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063159.nxs'),
                                       spectrum_size=spectrum_size, tof_bins=tof_bins)
ds

## Apply corrections to pixel positions

We apply some corrections to the detector pixel and monitor positions,
as the geometry stored in the file is inaccurate.

In [ ]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m

In [ ]:
ds.coords["pixel_width"] = 0.0035 * sc.units.m
ds.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

for key in ds:
    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

# Now shift pixels positions to get the correct beam center
ds.coords['position'].fields.x += x_offset
ds.coords['position'].fields.y += y_offset

## Mask bad pixels

We mask the edges of the detector, which are usually noisy.
We also mask the region close to the center of the beam,
so as to not include saturated pixels in our data reduction.

In [ ]:
mask_edges = (
    (sc.abs(ds.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(ds.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    ds.coords['position'].fields.x**2 +
    ds.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

We can inspect that the coordinate corrections and masking were applied correctly by opening the instrument view.

In [ ]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

## Use Q1D workflow

We now reduce the sample and the background measurements to `Q` using the `sans.q1d` workflow.

In that process,
the intensity as a function of `Q` is normalized using the direct measurement and direct beam function.

In [ ]:
sample_q = sans.q1d(data=ds['sample'],
        data_incident_monitor=ds['sample'].attrs["monitor2"].value,
        data_transmission_monitor=ds['sample'].attrs["monitor4"].value,
        direct_incident_monitor=ds['direct'].attrs["monitor2"].value,
        direct_transmission_monitor=ds['direct'].attrs["monitor4"].value,
        direct_beam=direct_beam,
        wavelength_bins=wavelength_bins,
        q_bins=q_bins,
        monitor_background_threshold=sc.scalar(30.0, unit='counts'))
sample_q.plot()

In [ ]:
background_q = sans.q1d(data=ds['background'],
        data_incident_monitor=ds['background'].attrs["monitor2"].value,
        data_transmission_monitor=ds['background'].attrs["monitor4"].value,
        direct_incident_monitor=ds['direct'].attrs["monitor2"].value,
        direct_transmission_monitor=ds['direct'].attrs["monitor4"].value,
        direct_beam=direct_beam,
        wavelength_bins=wavelength_bins,
        q_bins=q_bins,
        monitor_background_threshold=sc.scalar(30.0, unit='counts'))
background_q.plot()

We are now in a position to subtract the background from the sample measurement:

In [ ]:
result = sample_q - background_q
result

In [ ]:
fig1, ax1 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(result, ax=ax1[0])
sc.plot(result, norm='log', ax=ax1[1])
fig1

## Wavelength bands

It is often useful to process the data in a small number (~10) of separate wavelength bands.

This can be achieved by requesting 10 bands from the `q1d` workflow via the `number_of_wavelength_bands` argument.

In [ ]:
nbands = 10

sample_slices = sans.q1d(data=ds['sample'],
    data_incident_monitor=ds['sample'].attrs["monitor2"].value,
    data_transmission_monitor=ds['sample'].attrs["monitor4"].value,
    direct_incident_monitor=ds['direct'].attrs["monitor2"].value,
    direct_transmission_monitor=ds['direct'].attrs["monitor4"].value,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    number_of_wavelength_bands=nbands,
    monitor_background_threshold=sc.scalar(30.0, unit='counts'))

background_slices = sans.q1d(data=ds['background'],
    data_incident_monitor=ds['background'].attrs["monitor2"].value,
    data_transmission_monitor=ds['background'].attrs["monitor4"].value,
    direct_incident_monitor=ds['direct'].attrs["monitor2"].value,
    direct_transmission_monitor=ds['direct'].attrs["monitor4"].value,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    number_of_wavelength_bands=nbands,
    monitor_background_threshold=sc.scalar(30.0, unit='counts'))

result_slices = sample_slices - background_slices
result_slices

In [ ]:
collapsed = sc.collapse(result_slices, keep='Q')

fig2, ax2 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(collapsed, ax=ax2[0])
sc.plot(collapsed, norm='log', legend=False, ax=ax2[1])
fig2